### Importing necessary libraries

In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

#Date time
import datetime as dt
from datetime import timedelta, date

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd
import geopandas

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier
import torch


# Planetary Computer Tools
import pystac
import pystac_client
import stackstac
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
import contextily
pc.settings.set_subscription_key('b0067a12405d4fd4a4cc82d28869d9bc')

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()

#Scaling using Dask
import dask_gateway
import dask
import xarray as xr
from dask.distributed import Client
import dask.array as da
import dask.dataframe as dd

In [2]:
crop_presence_data = pd.read_csv("./Data/Crop_Location_Data_20221201.csv")
crop_presence_data.head()

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


In [3]:
df = pd.read_csv('./Data/temp/rvi-may.csv')

### Creating bounding boxes from the given coordinate

In [4]:
def calculate_bbox(lat_long, box_size_deg=0.0004):
    lat_long=lat_long.replace('(','').replace(')','').replace(' ','').split(',')
    
    min_lon = float(lat_long[1]) - box_size_deg/2
    min_lat = float(lat_long[0])- box_size_deg/2
    max_lon = float(lat_long[1]) + box_size_deg/2
    max_lat = float(lat_long[0]) + box_size_deg/2
    
    return min_lon, min_lat, max_lon, max_lat

#### Defining variables 

In [5]:
time_of_interest = '2021-11-01/2022-03-31'
resolution = 10  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for crs=4326 

In [6]:
rvi_df = pd.DataFrame() 
bbox = []
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    bbox.append(calculate_bbox(coordinates))

bbox_data = pd.DataFrame(bbox,columns =['min_lon', 'min_lat', 'max_lon', 'max_lat'])

100%|████████████████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 369162.74it/s]


#### Dask

### New approach

In [7]:
bbox_data

,min_lon,min_lat,max_lon,max_lat
0,105.251435,10.323527,105.251835,10.323927
1,105.278234,10.322164,105.278634,10.322564
2,105.252343,10.321256,105.252743,10.321656
3,105.250980,10.323981,105.251380,10.324381
4,105.273692,10.324436,105.274092,10.324836
...,...,...,...,...
595,105.673413,10.013743,105.673813,10.014143
596,105.673413,10.013289,105.673813,10.013689
597,105.673413,10.012835,105.673813,10.013235
598,105.673413,10.012380,105.673813,10.012780


In [8]:
def create_rvi_for_each_aoi(time_of_interest, bbox):
    #Creating catalog
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_of_interest)
    items = search.get_all_items()
    
    # Load the data using Open Data Cube
    data = stac_load(items,bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox, chunks={"time":10, "x": 307200, "y": 307200},
                     crs="EPSG:4326", resolution=scale)
    mean_aoi = data.mean(dim=['latitude','longitude'])
    data['vh'] = mean_aoi["vh"]
    data['vv'] = mean_aoi["vv"]
    dop = (mean_aoi.vv / (mean_aoi.vv + mean_aoi.vh))
    m = 1 - dop
    data['rvi'] = (np.sqrt(dop))*((4*mean_aoi.vh)/(mean_aoi.vv + mean_aoi.vh))
    return data

In [8]:
for i in range(509,600):
    vh=[]
    vv=[]
    rvi=[]
    print(i)
    ds = create_rvi_for_each_aoi(time_of_interest,bbox_data.iloc[i])
    #ds_monthly= ds.groupby('time.month').std()
    vh= ds.vh.compute().to_series()
    vv= ds.vv.compute().to_series()
    rvi= ds.rvi.compute().to_series()
    frame = { 'VV': vv, 'VH': vh, 'RVI': rvi}
    vh_vv_rvi_data = pd.DataFrame(frame)
    vh_vv_rvi_data.to_csv('rvi.csv', mode='a', index=True, header=False)

509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599


In [72]:
vh_vv_rvi_data

,VV,VH,RVI
time,,,
2021-12-04 22:46:07.919581,0.161205,0.084049,1.111368
2021-12-05 11:11:54.609425,0.204640,0.041761,0.617824
2021-12-10 22:45:35.316746,0.107861,0.026061,0.698573
2021-12-16 22:46:07.511215,0.028466,0.003311,0.394425
2021-12-17 11:11:54.196354,0.027198,0.006252,0.674143
2021-12-22 22:45:34.433936,0.074212,0.006685,0.316586
2021-12-28 22:46:06.761481,0.043079,0.006129,0.466150
2021-12-29 11:11:53.438580,0.032929,0.006217,0.582673


In [17]:
def create_latlong_csv(lat_long, items):
    frame = { 'latitude_longitude': lat_long}
    lat_long_data = pd.DataFrame(frame)
    for i in range(items):
         lat_long_data.to_csv('rvi.csv', mode='a', index=False, header=False)

In [25]:
df1 = crop_presence_data['Class of Land'].to_frame()
print(df1)
for i in range(600):
    create_latlong_csv(df1.iloc[i], 9)

    Class of Land
0            Rice
1            Rice
2            Rice
3            Rice
4            Rice
..            ...
595      Non Rice
596      Non Rice
597      Non Rice
598      Non Rice
599      Non Rice

[600 rows x 1 columns]


#### Merging multiple csv files and creating a single file

In [128]:
import os
import glob
import pandas as pd
# merging the files
joined_files = os.path.join("./Data/Temp/", "rvi-*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)

In [129]:
final_rice_df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

In [130]:
final_rice_df.shape

(31800, 6)

In [131]:
final_rice_df.head()

,image_taken,VV,VH,RVI,location,Class_of_Land
0,10/03/2022,0.089866,0.032007,0.902072,"(10.323727047081501, 105.2516346045924)",Rice
1,11/03/2022,0.050158,0.029283,1.171599,"(10.323727047081501, 105.2516346045924)",Rice
2,22/03/2022,0.222246,0.070375,0.838371,"(10.323727047081501, 105.2516346045924)",Rice
3,23/03/2022,0.098687,0.037883,0.943192,"(10.323727047081501, 105.2516346045924)",Rice
4,10/03/2022,0.315821,0.049165,0.501215,"(10.322364360592521, 105.27843410554115)",Rice


In [132]:
y_cat = LabelEncoder().fit(final_rice_df['Class_of_Land'])
final_rice_df['Class_of_Land'] = y_cat.transform(final_rice_df['Class_of_Land'])

In [133]:
X_reg = final_rice_df.drop(['image_taken','location', 'Class_of_Land'], axis=1)
y_reg= final_rice_df['Class_of_Land']

In [134]:
X_reg= StandardScaler().fit_transform(X_reg)

In [135]:
X_train, X_test, y_train, y_test = train_test_split(X_reg, y_reg, test_size=0.30, random_state=2)

#### Creating Tabnet classifier model

In [136]:
classifier = TabNetClassifier(optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10,"gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='sparsemax', verbose=0,seed=42)

classifier.fit(X_train=X_train, y_train=y_train,
               patience=5,max_epochs=100,
               eval_metric=['auc'])

In [137]:
preds = classifier.predict(X_test)

In [138]:
(preds == y_test).sum()

8050

In [139]:
preds.shape

(9540,)

In [140]:
8050/9540

0.8438155136268344

### Creating Ensemble learning with cross validation and bagging

In [141]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score

In [142]:
scores = cross_val_score(DecisionTreeClassifier(), X_train, y_train, cv=5)

In [143]:
scores.mean()

0.7874213836477988

In [144]:
bagging_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, max_samples=0.8, oob_score=True, random_state=0)
score= cross_val_score(bagging_model, X_train, y_train, cv=5)
score.mean()

0.8366127583108716

In [145]:
from sklearn.ensemble import RandomForestClassifier
score= cross_val_score(RandomForestClassifier(), X_train, y_train, cv=5)
score.mean()

0.8360287511230908

In [146]:
dt_model = DecisionTreeClassifier(max_depth=4, min_samples_leaf= 0.16, random_state= 1)
bagging_model = BaggingClassifier(base_estimator= dt_model, n_estimators=100, max_samples=0.8, 
                                  oob_score=True, random_state=1, n_jobs=-1).fit(X_train, y_train)
y_pred = bagging_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test set accuracy: {:.2f}".format(acc))
print("OOB Accuracy: {:.2f}".format(bagging_model.oob_score_))

Test set accuracy: 0.76
OOB Accuracy: 0.77


### Ensemble Learning

In [147]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import VotingClassifier
lr = LogisticRegression(random_state=1)
knn = KNN()
dt = DecisionTreeClassifier(random_state=1)

classifiers = [
    ('Logistic Regression', lr),
    ('K Nearest Neighbours', knn),
    ('Classification Tree', dt)
] 

In [148]:
vc = VotingClassifier(estimators= classifiers)

In [149]:
vc.fit(X_train, y_train)
y_pred = vc.predict(X_test)

In [151]:
accuracy_score(y_test,y_pred)

0.8263102725366877

In [113]:
df = pd.read_csv('./Data/Temp/rvi-aug.csv')

In [114]:
df['image_taken']

0       01/08/2022 00:00
1       02/08/2022 00:00
2       14/08/2022 00:00
3       25/08/2022 00:00
4       26/08/2022 00:00
              ...       
2995    01/08/2022 00:00
2996    02/08/2022 00:00
2997    14/08/2022 00:00
2998    25/08/2022 00:00
2999    26/08/2022 00:00
Name: image_taken, Length: 3000, dtype: object

In [115]:
df['image_taken'] = df.image_taken.str[:-6]

In [116]:
df['image_taken']

0       01/08/2022
1       02/08/2022
2       14/08/2022
3       25/08/2022
4       26/08/2022
           ...    
2995    01/08/2022
2996    02/08/2022
2997    14/08/2022
2998    25/08/2022
2999    26/08/2022
Name: image_taken, Length: 3000, dtype: object

In [117]:
df1 = pd.read_csv('./Data/Temp/rvi-aug.csv')

In [118]:
df1['image_date']= df['image_taken']

In [119]:
df1.to_csv('aug.csv', index=False)

#### Sample code to explore STAC items

In [23]:
@dask.delayed
def create_rvi_for_each_aoi(time_of_interest, bbox):
    #Creating catalog
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_of_interest)
    items = search.get_all_items()
    print(len(items))
    
    # Load the data using Open Data Cube
    data = stac_load(items,bands=["vv", "vh"], patch_url=pc.sign, chunks={"time":10, "x": 307200, "y": 307200},
                     crs="EPSG:4326", resolution=scale)
   
    return data

#### Creating lazy datasets from delayed objects and calculating mean scattering

In [24]:
#Defining lazy null datasets
lazy_datasets = []

#Creating list of all lazy datasets
for i in range(600):
    ds = create_rvi_for_each_aoi(time_of_interest,bbox_data.iloc[i])
    lazy_datasets.append(ds)
datasets = dask.compute(*lazy_datasets)

#### Calculating mean scattering and radar vegetation index

#### Converting xarray dataarray to series

In [108]:
def convert_xarraydataset_to_dataframe(ddf):
    mean_aoi = ddf.groupby(['time'])['latitude', 'longitude'].mean()
    #mean_aoi = ddf.mean(dim=['latitude','longitude']).compute()
    #print('Mean polarization', mean_aoi)

    #Calculating mean vv and vh of our area of interest
    vh = mean_aoi["vh"]
    vv = mean_aoi["vv"]
    #print('Vertical-vertical polarisation', vv)

    #print('Vertical-horizontal polarisation', vh)
    # Calculate RVI
    dop = (mean_aoi.vv / (mean_aoi.vv + mean_aoi.vh))
    m = 1 - dop
    rvi = (np.sqrt(dop))*((4*mean_aoi.vh)/(mean_aoi.vv + mean_aoi.vh))
    
    return rvi, vv, vh

#### Converting xarray dataset to dask dataframe

In [131]:
mean_aoi = DS.mean(dim=['latitude','longitude'])
#print('Mean polarization', mean_aoi)

#Calculating mean vv and vh of our area of interest
vh = mean_aoi["vh"]
vv = mean_aoi["vv"]
#print('Vertical-vertical polarisation', vv)

#print('Vertical-horizontal polarisation', vh)
# Calculate RVI
dop = (mean_aoi.vv / (mean_aoi.vv + mean_aoi.vh))
m = 1 - dop
rvi = (np.sqrt(dop))*((4*mean_aoi.vh)/(mean_aoi.vv + mean_aoi.vh))

In [101]:
ddf

,latitude,longitude,time,spatial_ref,vv,vh
npartitions=37,,,,,,
0,float64,float64,datetime64[ns],int32,float32,float32
32940,...,...,...,...,...,...
...,...,...,...,...,...,...
1185840,...,...,...,...,...,...
1218779,...,...,...,...,...,...


In [ ]:
vh_vv_data = pd.DataFrame(list(zip(vh_series,vv_series,rvi_series)),columns = ["vv_list","vh_list","rvi_list"])

#### Starting Dask distribution

#### Lazily append all datasets

In [65]:
lazy_datasets = []
for i,item in bbox_data.iterrows():
    print(i)
    ds = create_rvi_for_each_aoi(time_of_interest, item)
    lazy_datasets.append(ds)

datasets = dask.compute(*lazy_datasets)

#Concatening 
DS = xr.concat(datasets, dim='time')

#Creating mean VV, VH Scattering
# Calculate the mean of the data across the sample region
mean_aoi = DS.mean(dim=['latitude','longitude'])
#print('Mean polarization', mean_aoi)

#Calculating mean vv and vh of our area of interest
vh = mean_aoi["vh"]
vv = mean_aoi["vv"]
#print('Vertical-vertical polarisation', vv)

#print('Vertical-horizontal polarisation', vh)
# Calculate RVI
dop = (mean_aoi.vv / (mean_aoi.vv + mean_aoi.vh))
m = 1 - dop
rvi = (np.sqrt(dop))*((4*mean_aoi.vh)/(mean_aoi.vv + mean_aoi.vh))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2023-03-12 13:29:41,701 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
